In [1]:
#!pip install tqdm
#!pip install evaluate
#!pip install accelerate -U
#!pip install  transformers datasets scipy torch
#!pip install nltk

In [2]:
#!pip install evaluate
import pandas as pd
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
#from transformers import AutoTokenizer
import os
import pandas as pd
from transformers import TextClassificationPipeline

from transformers import AutoTokenizer
from transformers import TextClassificationPipeline
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback, IntervalStrategy
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq

#from sklearn.model_selection import train_test_split
from datasets import Dataset
from datasets import concatenate_datasets
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

import torch
import gc
from numba import cuda

In [3]:
## important settings
run = 4
runs = 1
epochs = 5
## model used for experiments
model_name="google/flan-t5-base"
model_max_length = 512

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# Load tokenizer of FLAN-t5-base
#tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
constructs = ['sexism']
construct_names = [
    'sexism',
    ]
perplexity = 10 # 1, 5, 10, 15, 20


reverse_label_encode = {'sexism' : {"1" : 'sexist', "0" : 'non-sexist'},
                        'hatespeech' : {"1" : 'hate', "0" : 'not hate'}
                       }
            
label_encode = {'hatespeech': {'hate' : "1", 'not hate': "0"},
                    'sexism' : {'sexist' : "1", 'non-sexist': "0"}
                   }


In [6]:
classifiers = [
               # ('MNB', MultinomialNB),
               #('Linear SVM', LinearSVC),
               # ('LR', LogisticRegression),
               # ('RF', RandomForestClassifier),
               ('transformer', 'transformer')
]

features = ['_TF-IDF', '_fasttext']

modes = {
         'OG' : ('original_text', 'original_label'),
         'CAD' : ('counterfactual_text', 'counterfactual_label'),
         'aCAD' : ('polyjuice', 'polyjuice_label'),
         'aCAD_GPT' : ('chatgpt', 'chatgpt_label'),
         'aCAD_FT' : ('flant5', 'flant5_label'),
         'CAD_mixed' : ('mixed_cad_text', 'mixed_cad_label'),   
    
}

In [7]:
paired_data = {}

for construct in construct_names:
    train_data_path = '%s' %construct
    # paired_data[construct] = pd.read_csv(train_data_path + '/paired_adv_cad_GPT_%d.csv' %(perplexity), sep = '\t')
    paired_data[construct] = pd.read_csv(train_data_path + '/paired_cads_mixed.csv', sep = '\t')
    
    paired_data[construct]['polyjuice_id'] = [str(i)+'p' for i in paired_data[construct]['original_id']]
    paired_data[construct]['chatgpt_id'] = [str(i)+'gpt' for i in paired_data[construct]['original_id']]
    paired_data[construct]['flant5_id'] = [str(i)+'ft' for i in paired_data[construct]['original_id']]
    paired_data[construct]['mixed_id'] = [str(i)+'m' for i in paired_data[construct]['original_id']]

In [8]:
## print loaded data

for construct in construct_names:
    print(construct)
    print(paired_data[construct].groupby('original_label').size())
    print()
    print(paired_data[construct].groupby('counterfactual_label').size())
    print()
    print(paired_data[construct].groupby('polyjuice_label').size())
    print()
    print(paired_data[construct].groupby('chatgpt_label').size())
    print()
    print(paired_data[construct].groupby('flant5_label').size())
    print()
    print(paired_data[construct].groupby('mixed_cad_label').size())
    print('---------------------------------------------------------')

sexism
original_label
non-sexist    1610
sexist        1244
dtype: int64

counterfactual_label
MAR            596
NMAR          1610
non-sexist     648
dtype: int64

polyjuice_label
MAR            553
NMAR          1610
non-sexist     691
dtype: int64

chatgpt_label
NMAR          1610
non-sexist    1244
dtype: int64

flant5_label
MAR            103
NMAR          1610
non-sexist    1141
dtype: int64

mixed_cad_label
MAR            375
NMAR          1610
non-sexist     869
dtype: int64
---------------------------------------------------------


In [9]:
label_reverse = {'hate' : 'not hate', 'not hate' : 'hate', 'MAR' : 'MAR',
                 'sexist' : 'non-sexist', 'non-sexist' : 'sexist'}

In [10]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
#nltk.download("punkt")

# Metric
metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]
    
    ## non-expected outputs are mapped to 0 
    preds = ["0" if ((x != "0") and (x != "1")) else x for x in preds]
    
    #print(preds)
    #print(labels)
    return preds, labels



In [11]:
import random
n = random.randint(0,1000)

In [12]:
from transformers import set_seed



In [13]:

## main optimization function
def train_model(data_, construct = 'hatespeech',
                features = 'TF-IDF', classifier_type = None,
                mode = 'OG', run = 0):
    
    ## reset
    #trainer = None
    #results = None
    #tokenizer = None
    #metric = None
    #from transformers import set_seed

    set_seed(run)
    
    ## new metrics and tokenizer are initialized for each hyperparameter run
    metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    
    
    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {k: round(v * 100, 4) for k, v in result.items()}
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        return result
    
    def preprocess_function(sample, padding="max_length"):
        max_target_length=3
        max_source_length = 512
        #print(sample["label"])

        # add prefix to the input for t5
        inputs = [item for item in sample["text"]]

        # tokenize inputs
        model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

        # Tokenize targets with the `text_target` keyword argument
        labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length":
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    data = data_.copy()
    # step 1. stratify and equalize classes (by original labels)
    g = data.groupby(modes['OG'][1])
    training_data = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))
    training_data = training_data.sample(frac=1)
    #print(len(training_data))
    #print(training_data.head()["original_text"])
    
    ## DIFFERENT SETTINGS FOR DATA
    if mode == 'OG':
        training_X = training_data[modes['OG'][0] + features].values
        training_y = training_data[modes['OG'][1]].values
        
        training_data_datamap = [training_data[modes['OG'][0].split('_')[0] + '_id'].values,
                                 training_data[modes['OG'][0]].values,
                                 training_data[modes['OG'][1]].values,
                                 ['original'] * len(training_data[modes['OG'][1]])
                                ]
    else:
        # step 2. keep pos class as is
        pos_data = training_data[training_data[modes['OG'][1]] == reverse_label_encode[construct]["1"]]
        training_X = list(pos_data[modes['OG'][0] + features].values) #adding the OGs
        training_y = list(pos_data[modes['OG'][1]].values)
            
        # Step 3: drop half of the negative data (non-sexist, non-hate)
        neg_data = training_data[training_data[modes['OG'][1]] == reverse_label_encode[construct]["0"]]
        neg_sample_len = len(neg_data)
        neg_data = neg_data.sample(n = neg_sample_len//2)
        training_X.extend(list(neg_data[modes['OG'][0] + features].values)) #adding the OGs
        training_y.extend(list(neg_data[modes['OG'][1]].values))
            
        # replace with that much counterfactual
        neg_cad_data = pos_data#.sample(n = neg_sample_len//2)
        # drop the missing CADs (MAR)
        neg_cad_data = neg_cad_data[neg_cad_data[modes[mode][1]] != 'MAR']
        neg_cad_data = neg_cad_data.sample(n = neg_sample_len//2)
        training_X.extend(list(neg_cad_data[modes[mode][0] + features].values)) #adding the CADs
        training_y.extend(list(neg_cad_data[modes[mode][1]].values))
            
        ids = list(pos_data[modes['OG'][0].split('_')[0] + '_id'].values)
        ids.extend(list(neg_data[modes['OG'][0].split('_')[0] + '_id'].values))
        ids.extend(list(neg_cad_data[modes[mode][0].split('_')[0] + '_id'].values))
        
        texts = list(pos_data[modes['OG'][0]].values)
        texts.extend(list(neg_data[modes['OG'][0]].values))
        texts.extend(list(neg_cad_data[modes[mode][0]].values))
        
        labels = list(pos_data[modes['OG'][1]].values)
        labels.extend(list(neg_data[modes['OG'][1]].values))
        labels.extend(list(neg_cad_data[modes[mode][1]].values))
        
        mode_lists = ['original'] * (len(pos_data) + len(neg_data))
        mode_lists.extend([modes[mode][1].split('_')[0]] * len(neg_cad_data))
        
        training_data_datamap = [ids, texts, labels, mode_lists]
        
    training_data_datamap_df = pd.DataFrame(training_data_datamap).T
    training_data_datamap_df.columns = ['id', 'text', 'label', 'data_type']
    print('FINAL DIST: ', training_data_datamap_df.groupby('label').size())
    
    

    train_data = []
    
    for n, text in enumerate(training_X):
        train_data.append((text, label_encode[construct][training_y[n]]))
    train_df = pd.DataFrame(train_data)
    train_df.columns = ["text", "labels"]
    train_df["label"] = train_df["labels"]
    dataset = Dataset.from_pandas(train_df)
    
    #print(train_df.tail())

    tokenized_datasets = dataset.map(preprocess_function, batched=True)
    #print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")
    #tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.class_encode_column("label")

    ## create splits
    split = tokenized_datasets.train_test_split(test_size = 0.2, stratify_by_column="label")
    
    
    split["train"]=split["train"].remove_columns("label")
    split["train"]=split["train"].remove_columns("text")
    split["test"]=split["test"].remove_columns("label")
    split["test"]=split["test"].remove_columns("text")
    #print(split["train"]["input_ids"][0])
    
    ## init model
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    # we want to ignore tokenizer pad token in the loss
    label_pad_token_id = -100
    
    # Data collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8
    )
   
    ## setup training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir='_outflant5/_out_'+str(construct)+"_"+str(mode)+"_"+str(run),
        #evaluation_strategy="epoch",
        learning_rate=5e-4,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        
        evaluation_strategy="no",
        save_strategy="epoch",
        save_total_limit=2,
        num_train_epochs=epochs,
        load_best_model_at_end=False,
        weight_decay=0.001,
        #save_strategy="epoch",
        #do_predict=True,
        logging_steps=200,
        logging_dir="logs",
        skip_memory_metrics=True,
        report_to="none",
        predict_with_generate=True,
        seed=run

    )
    
    
    ## setup trainer
    trainer =  Seq2SeqTrainer(
        args=training_args,
        train_dataset=split["train"],
        eval_dataset=split["test"],
        compute_metrics=compute_metrics,
        model=model,
        data_collator=data_collator
    )
    
    
    trainer.train()
    print(str(construct)+"_"+str(mode)+"_"+str(run))
    print(trainer.evaluate())

    trainer.save_model("_models/flant5_"+str(construct)+"_"+str(mode)+"_"+str(run))
    
    del training_args
    
    del model
    #gc.collect()
    del tokenizer
    #gc.collect()
    del data_collator
    #gc.collect()
    del trainer
    #gc.collect()
    del split
    gc.collect()
    #data_collator = None
    #model = None
    #trainer = None
    #results = None 
    #tokenizer = None
    #metric = None
    #split= None
    
   # gc.collect()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()
    
    return "_models/flant5_"+str(construct)+"_"+str(mode)+"_"+str(run)

In [14]:
model = None
trainer = None
results = None 
tokenizer = None
metric = None
split= None
torch.cuda.empty_cache()

In [15]:
#gc.get_objects()

In [16]:
all_models = []
## iterate over constructs
for construct in constructs:
    model = {}
    model['construct'] = construct
    ## oterate over modes
    for mode in modes:
        print(mode)
        model['mode'] = mode
        
        ## iterate over classifier
        for classifier_name, classifier_type in classifiers:
            model['classifier_name'] = classifier_name
            ## iterate over number of runs
            #for run in range(runs):
            model['run'] = run
            if classifier_type == 'transformer':
                    model['feature'] = ''
                
            model['classifier'] = train_model(paired_data[construct], construct,
                                                features = model['feature'], classifier_type = classifier_type,
                                                mode = mode, run = run)
            #device = cuda.get_current_device()
            #device.reset()
            all_models.append(model.copy())

OG
FINAL DIST:  label
non-sexist    1244
sexist        1244
dtype: int64


Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2488 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
200,0.228700
400,0.064800
600,0.014600


...

Casting to class labels:   0%|          | 0/2488 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
200,0.270500
400,0.155600
600,0.100600


sexism_CAD_mixed_4


{'eval_loss': 0.42411237955093384, 'eval_accuracy': 73.2932, 'eval_f1': 74.9529, 'eval_precision': 70.5674, 'eval_recall': 79.9197, 'eval_gen_len': 2.433734939759036, 'eval_runtime': 21.9485, 'eval_samples_per_second': 22.689, 'eval_steps_per_second': 1.458, 'epoch': 5.0}


In [17]:
test_sets = ['in_domain', 'out_of_domain', 
             'out_of_domain_2', 'out_of_domain_3', 
             #'out_of_domain_4', 
             'hatecheck']

In [18]:
test_set_data = {}

for construct in construct_names:
    test_set_data[construct] = {}
    for test_set in test_sets:
        test_path = '%s/test/%s.csv' %(construct, test_set)
        test_set_data[construct][test_set] = pd.read_csv(test_path, sep = '\t')

In [19]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, roc_curve, roc_auc_score, f1_score, accuracy_score, recall_score
from sklearn import metrics

In [20]:
all_models

[{'construct': 'sexism',
  'mode': 'OG',
  'classifier_name': 'transformer',
  'run': 4,
  'feature': '',
  'classifier': '_models/flant5_sexism_OG_4'},
 {'construct': 'sexism',
  'mode': 'CAD',
  'classifier_name': 'transformer',
  'run': 4,
  'feature': '',
  'classifier': '_models/flant5_sexism_CAD_4'},
 {'construct': 'sexism',
  'mode': 'aCAD',
  'classifier_name': 'transformer',
  'run': 4,
  'feature': '',
  'classifier': '_models/flant5_sexism_aCAD_4'},
 {'construct': 'sexism',
  'mode': 'aCAD_GPT',
  'classifier_name': 'transformer',
  'run': 4,
  'feature': '',
  'classifier': '_models/flant5_sexism_aCAD_GPT_4'},
 {'construct': 'sexism',
  'mode': 'aCAD_FT',
  'classifier_name': 'transformer',
  'run': 4,
  'feature': '',
  'classifier': '_models/flant5_sexism_aCAD_FT_4'},
 {'construct': 'sexism',
  'mode': 'CAD_mixed',
  'classifier_name': 'transformer',
  'run': 4,
  'feature': '',
  'classifier': '_models/flant5_sexism_CAD_mixed_4'}]

In [21]:
all_results = []

for model in all_models:
    result = model.copy()
    for test_set in test_sets:
        result['test_set'] = test_set
        test_data = test_set_data[model['construct']][test_set]
        construct = model['construct']
        
        if model['classifier_name'] == 'transformer':
            test_features = list(test_data['text'].values)
            
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            classifier = AutoModelForSeq2SeqLM.from_pretrained(model['classifier'])
            classifier.to('cuda')

            preds = []
            for x in test_features:
                inputs = tokenizer.encode_plus(x, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
                outputs = classifier.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=3, num_beams=1,do_sample=False)
                out = tokenizer.decode(outputs[0], skip_special_tokens=True)
                preds.append(out)
            
            
            preds,k = postprocess_text(preds,"")
    
            
            true = [label_encode[construct][x] for x in test_set_data[construct][test_set][construct].values]
       
      
        else:
            print("wrong way")

        
        scores = precision_recall_fscore_support(true, preds)
        test_data['%s_%s_%d_predictions' %(model['mode'], model['classifier_name'],
                                              model['run'])] = preds

        
        result['predictions'] = preds
        result['pos_precision'] = scores[0][0]
        result['neg_precision'] = scores[0][1]
        result['pos_recall'] = scores[1][0]
        result['neg_recall'] = scores[1][0]
        result['pos_f1'] = scores[2][0]
        result['neg_f1'] = scores[2][1]
        result['macro_f1'] = f1_score(true, preds, average = 'macro')
        result['micro_f1'] = f1_score(true, preds, average = 'micro')
        
        print(str(result['classifier']))
        print(str(result['test_set']))
        print("macro F1: " + str(result['macro_f1']))
        print("-------------------")
       
        classifier = None
        tokenizer = None

        torch.cuda.empty_cache()
    
        all_results.append(result.copy())

_models/flant5_sexism_OG_4
in_domain
macro F1: 0.8397730278185185
-------------------
_models/flant5_sexism_OG_4
out_of_domain
macro F1: 0.6073546991204692
-------------------
_models/flant5_sexism_OG_4
out_of_domain_2
macro F1: 0.6028796745521211
-------------------
_models/flant5_sexism_OG_4
out_of_domain_3
macro F1: 0.5881112600983974
-------------------
_models/flant5_sexism_OG_4
hatecheck
macro F1: 0.5552959343316334
-------------------
_models/flant5_sexism_CAD_4
in_domain
macro F1: 0.8186230926348672
-------------------
_models/flant5_sexism_CAD_4
out_of_domain
macro F1: 0.6387335414965596
-------------------
_models/flant5_sexism_CAD_4
out_of_domain_2
macro F1: 0.6091739219292868
-------------------
_models/flant5_sexism_CAD_4
out_of_domain_3
macro F1: 0.6025049157358632
-------------------
_models/flant5_sexism_CAD_4
hatecheck
macro F1: 0.5763764404609475
-------------------
_models/flant5_sexism_aCAD_4
in_domain
macro F1: 0.7818916841665708
-------------------
_models/flant5_

In [22]:
import pickle
with open('results_%d.pkl' %run, 'wb') as handle:
    pickle.dump(all_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
with open('sexism_results_%d.pkl' %run, "rb") as input_file:
    e = pickle.load(input_file)

In [24]:
all_results_df = pd.DataFrame(all_results)
all_results_df.to_csv("flant5_results_sexism_run_%d.csv" %run)

In [25]:
all_results_df.head()

,construct,mode,classifier_name,run,feature,classifier,test_set,predictions,pos_precision,neg_precision,pos_recall,neg_recall,pos_f1,neg_f1,macro_f1,micro_f1
0,sexism,OG,transformer,4,,_models/flant5_sexism_OG_4,in_domain,"[0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, ...",0.937943,0.756061,0.766667,0.766667,0.843700,0.835846,0.839773,0.839869
1,sexism,OG,transformer,4,,_models/flant5_sexism_OG_4,out_of_domain,"[1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, ...",0.614230,0.612409,0.705000,0.705000,0.656492,0.558217,0.607355,0.613504
2,sexism,OG,transformer,4,,_models/flant5_sexism_OG_4,out_of_domain_2,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.956436,0.249254,0.742316,0.742316,0.835881,0.369878,0.602880,0.739588
3,sexism,OG,transformer,4,,_models/flant5_sexism_OG_4,out_of_domain_3,"[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, ...",0.845180,0.354842,0.625710,0.625710,0.719071,0.457151,0.588111,0.629750
4,sexism,OG,transformer,4,,_models/flant5_sexism_OG_4,hatecheck,"[0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...",0.347490,0.816000,0.661765,0.661765,0.455696,0.654896,0.555296,0.577603


In [26]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [27]:
all_results_df.groupby(['mode', 'test_set', 'classifier_name']).mean()[['macro_f1', 'micro_f1']].unstack()

/tmp/ipykernel_196696/1615460453.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_results_df.groupby(['mode', 'test_set', 'classifier_name']).mean()[['macro_f1', 'micro_f1']].unstack()


macro_f1    micro_f1
classifier_name           transformer transformer
mode      test_set                               
CAD       hatecheck          0.576376    0.642436
          in_domain          0.818623    0.818627
          out_of_domain      0.638734    0.642026
          out_of_domain_2    0.609174    0.749657
          out_of_domain_3    0.602505    0.648550
CAD_mixed hatecheck          0.614798    0.664047
          in_domain          0.814423    0.814542
          out_of_domain      0.638116    0.640570
          out_of_domain_2    0.647582    0.795728
          out_of_domain_3    0.618355    0.664600
OG        hatecheck          0.555296    0.577603
          in_domain          0.839773    0.839869
          out_of_domain      0.607355    0.613504
          out_of_domain_2    0.602880    0.739588
          out_of_domain_3    0.588111    0.629750
aCAD      hatecheck          0.356126    0.359528
          in_domain          0.781892    0.783497
          out_of_domain      0.575228    0.598370
          out_of_domain_2    0.611231    0.798627
          out_of_domain_3    0.601993    0.700700
aCAD_FT   hatecheck          0.558820    0.571709
          in_domain          0.809880    0.812092
          out_of_domain      0.611488    0.619325
          out_of_domain_2    0.607798    0.784592
          out_of_domain_3    0.588841    0.677400
aCAD_GPT  hatecheck          0.587471    0.626719
          in_domain          0.807899    0.808007
          out_of_domain      0.629167    0.634168
          out_of_domain_2    0.625666    0.788101
          out_of_domain_3    0.615055    0.664350

In [28]:
 torch.cuda.empty_cache()